<a href="https://colab.research.google.com/github/SupermarketAutomationAI/baseline_model_SVM/blob/main/baseline_model_SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello WORLD!");
print("Integrated with Github!")

Hello World
Integrated with Github!
